In [73]:
# The aim of this script is to pull necessary information 
# from a graph (nodes, edges) such as start and end nodes of an edge 
# to compare these and determine the connection on that edge (open or closed)
# After the open/closed data is put into the nodes' nested dictionary
# to define the module type 

In [74]:
#initialisation
import networkx as nx
import itertools
import json 
import re, requests

In [75]:
#definitions

#gives adjacency dictionary
edge_dict={}
def create_edge_dict(graph):
    for i, n in G.adjacency():
        # print("i is",i)
        # print("n is",n)
        edge_dict[i] = list(n)
    return edge_dict

# gives the parent of the value in a nested dictionary
def find_key(d, value):
    start_list =[]
    for i in value:
        # print("i is", i)
        for k,v in d.items():
            # print("k and v is", k,v)
            if v['name'] == i:
                # print("k is", k)
                start_list.append(i)
    return start_list

#gives the value of tag key of a node
def retrieve_tag(list):
    tag_list = []
    for i in list:
        # print ("i is", i)
        tag = nodes[i]['tag']
        # print(tag)
        tag_list.append(tag)
    return (tag_list)

#creates unique combinations from given string of available combinations
def create_combinations(combination_str):
    permutation_list = list(set(itertools.permutations(str(combination_str), 4)))
    p_list = []
    for i in range(len(permutation_list)):
            a = [int(x) for x in permutation_list[i]]
            p_list.append(a)
    return p_list

In [76]:
#Import the nodes data from github
"""# Import from online repository
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Allocation/211024_nodes_occupied.txt"
resp = requests.get(url)
nodes = json.loads(resp.text)
#converts keys from str to int
nodes = {int(k) : v for k,v in nodes.items()}"""

#retrieve from folder directly
file_nodes_occupied = "/Users/junwenloo/OneDrive/2020 ONWARDS/06_Delft/15_Y2Q1/01_Earthy/03_Scripts/Earthy4.2/0_Configuration/Allocation/211024_nodes_occupied.txt"
with open(str(file_nodes_occupied)) as read_json_file:
    nodes = json.load(read_json_file)
    #converts keys from str to int
    nodes = {int(k) : v for k,v in nodes.items()}

# Import edges data
"""# Import from online repository
url = "https://raw.githubusercontent.com/erengozdeanil/Earthy4.2/main/0_Configuration/Foundation/edges.txt"
resp = requests.get(url)
edges = json.loads(resp.text)
#converts nested lists into a list of tuples
edges = [tuple(i) for i in edges]"""

#retrieve from folder directly
file_edges = "/Users/junwenloo/OneDrive/2020 ONWARDS/06_Delft/15_Y2Q1/01_Earthy/03_Scripts/Earthy4.2/0_Configuration/Foundation/edges.txt"
with open(str(file_edges)) as read_json_file:
    edges = json.load(read_json_file)
    #converts nested lists into a list of tuples
    edges = [tuple(i) for i in edges]
#Number of nodes in a layer
nodes_count_1 = 580


In [77]:
# Draw the graph
# It should be noted that we only draw the graph to visualise the connections 
# and to use some fuctions that can only be used with graphs for the sake of efficiency
# There is no position of the nodes
G = nx.Graph()
G.add_edges_from(edges)
# pos = nx.spring_layout(G)

# nx.draw_networkx(G, pos)

# Get adjacency dictionary
edge_dict = create_edge_dict(G)
#print(edge_dict)

In [78]:
# create a dictionary of open/closed lists attributed to each node
edge_dict_connection={}
# print(edge_dict)
# determine the connection conditions based on the following criteria ordered by priority 
# the condition that 'tags are different' is at the end because if the tag is x or None or a street tag, the tags are still different
for k,v in edge_dict.items():
    edge_connection_list=[]
    for i in v:
        # if two neighboring nodes have the same tag then the connection is open(0)
        if nodes[k]["tag"]==nodes[i]["tag"]:
            same_shop = 0
            edge_connection_list.append(same_shop)
        # if one of the nodes is a street node(local street 0, middle street 1, main street 2) then the connection is open(0)
        elif (nodes[k]["tag"]==0) or (nodes[i]["tag"]==0) or (nodes[k]["tag"]==1) or (nodes[i]["tag"]==1) or (nodes[k]["tag"]==2) or (nodes[i]["tag"]==4 or (nodes[i]["tag"]==4)):
            street_shop = 0
            edge_connection_list.append(street_shop)
        # if one of the nodes is out of the set boundary (x) or if there is no shop on it then the connection is closed and there will be a buttress(2) 
        elif (nodes[k]["tag"]=="x") or (nodes[i]["tag"]=="x") or (nodes[k]["tag"]==None) or (nodes[i]["tag"]==None):
            nothing = 2
            edge_connection_list.append(nothing)
        # in case of any other condition such as tags being different(shops owned by different people) then the connection is closed(1)
        else:
            edge_oc_closed = (1)
            edge_connection_list.append(edge_oc_closed)
        edge_dict_connection[k]=edge_connection_list
# print(edge_dict_connection)


In [79]:
# #adds the connection data (eg:[0,0,0,0]) to nodes dictionary
# #nodes is a dictionary
for k in nodes:
    nodes[k]["connection"]=None
for k,v in edge_dict_connection.items():
    nodes[k]["connection"]=[]
    nodes[k]["connection"]=v

In [80]:
#create a list of types having the info of connections using the create_combination definition 
# which iterates every possible combination of 4 numbers and gathers them in a list
# create a list of possible floor conditions (0=nothing on top, 1=building on top)

#connection conditions
type_10 = create_combinations('0000')
type_20 = create_combinations('1000')
type_21 = create_combinations('2000')
type_30 = create_combinations('1100')
type_31 = create_combinations('1200')
type_32 = create_combinations('2200')
type_40 = create_combinations('1110')
type_41 = create_combinations('1120')
type_42 = create_combinations('1210')
type_43 = create_combinations('2210')
type_44 = create_combinations('2120')
type_45 = create_combinations('2220')
type_50 = create_combinations('1100')
type_51 = create_combinations('2001')
type_52 = create_combinations('2200')

# Create a seperate dictionary holding the module type name 
# and possible combinations of open/closed 
# for the exceptional modules which should be assigned seperately 
# since the number of closed and open edges are the same for both of these mpdules(30 and 50)

# Module types 30 and 50
types = {30.1: type_30, 31.1: type_31, 32.1:type_32}
counter_types = {30.1: 50.1, 31.1: 51.1, 32.1:52.1}

# Module types 41 and 42 [1012]
types_41 = {41.1: type_41}
counter_types_41 = {41.1: 42.1}

# Module types 43 and 44 [2021]
types_43 = {43.1: type_43}
counter_types_43 = {43.1: 44.1}

# Type_60 = [street],[0]
# Type_70 = Stair

#floor conditions
# F1 = (nodes[i]["floor"]==0)
# F2 = (nodes[i]["floor"]==1)

In [81]:
# exception 30&50
# create a dictionary containing closed edges
edge_dict_closed={}
for k,v in edge_dict_connection.items(): 
    for i in v:
        # for module types 30 and 50, there are two 0s and two 1s or 2s in the list
        # therefore we can check the count of 0s instead of checking 1s or 2s
        # if there are two 0s then, these nodes are added to the dictionary we will work with on the next steps 
        closed_edge_count=v.count(0)
        if closed_edge_count == 2:
            edge_dict_closed[k]=edge_dict[k] 

# from this dictionary create a new dictionary 
# holding only the nodes which have closed connection(1 or 2) to the key node
closed_edges={}
for k,v in edge_dict_closed.items():
    closed_nodes=[]
    for no, m in enumerate(edge_dict_connection[k]):
        # print(no)
        if (m==1) or (m==2):
            closed_nodes.append(edge_dict_closed[k][no])
    closed_edges[k]=closed_nodes

In [82]:
# assign a new empty attribute,module type, to the nodes
for k in nodes:
    nodes[k]["module"]= None

# type 30 and type 50 have an exception as they both have the same list of numbers
# but the position of open&closed edges differ between these two types
# start the if statement by asking whether the closed edge neighbors of the node(which we are assigning a module) have a neighbor in common
# if the nodes that have a closed(1) connection with the parent node have a neighbor in common module type is from type 30 family
# if it is not a neighbor then it should be a module from type 50 family
# Check also for some other conditions which may apply to type 30 and 50 while doing the neighbor check

for k,v in closed_edges.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # find the neighbors of the nodes connected to the node we are assigning a module to
        # put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types.items():
            if (nodes[k]["connection"] in tc):
                # check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # if there are no common neighbors of neighbors then closed edges are on the opposite sides (type 50)
                    nodes[k]["module"] = counter_types[ti]          

In [83]:
# exception 41&42
# make a list of keys of exceptional modules(41&42)
module_41_42_list=[]
for k in nodes:
    if nodes[k]["connection"] in type_41:
       module_41_42_list.append(k)

reinforced_dict_41={}
for i in module_41_42_list:
    for k in edge_dict_connection:
        reinforced_list_41=[]
        for no,t in enumerate(edge_dict_connection[i]):
             if t==1:
                reinforced_list_41.append(edge_dict[i][no])  
                reinforced_dict_41[i] = reinforced_list_41


for k,v in reinforced_dict_41.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # find the neighbors of the nodes connected to the node we are assigning a module to
        # put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types_41.items():
            if (nodes[k]["connection"] in tc):
                # check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # if there are no common neighbors of neighbors then closed edges are on the opposite sides (type 43)
                    nodes[k]["module"] = counter_types_41[ti] 

In [84]:
# exception 43&44
# make a list of keys of exceptional modules(43&44)
module_43_44_list=[]
for k in nodes:
    if nodes[k]["connection"] in type_43:
       module_43_44_list.append(k)

reinforced_dict_43={}
for i in module_43_44_list:
    for k in edge_dict_connection:
        reinforced_list_43=[]
        for no,t in enumerate(edge_dict_connection[i]):
             if t==2:
                reinforced_list_43.append(edge_dict[i][no])  
                reinforced_dict_43[i] = reinforced_list_43


for k,v in reinforced_dict_43.items():
    if (nodes[k]["type"]=="1") or (nodes[k]["type"]=="3") or (nodes[k]["type"]=="2") and (nodes[k]["type"]!=None):
        neigh_neigh = []
        if len(v) < 2:
            continue

        # find the neighbors of the nodes connected to the node we are assigning a module to
        # put them into a set
        for i in v:
            neigh_neigh.append(set(G.neighbors(i)))
        nn0, nn1 = tuple(neigh_neigh)

        # check for common nodes in nn0 and nn1
        intersection = nn0.intersection(nn1)
        for ti, tc in types_43.items():
            if (nodes[k]["connection"] in tc):
                # check for 2 intersections because first intersection is the node we are trying to define the module of
                # second intersection is the neighbour of the neighbouring node to the node we are trying to define the module of
                if len(intersection)==2:
                    nodes[k]["module"] = ti 
                else:
                    # if there are no common neighbors of neighbors then closed edges are on the opposite sides (type 44)
                    nodes[k]["module"] = counter_types_43[ti] 

In [85]:
#assigns module types to nodes according to connections except (30 and 50 family)
for i in nodes:
    if ((nodes[i]["type"]=="1") or (nodes[i]["type"]=="3") or (nodes[i]["type"]=="2")) and (nodes[i]["type"]!=None):
        if (nodes[i]["connection"] in type_10 ) :
            nodes[i]["module"] = 10.1
        elif (nodes[i]["connection"] in type_20) :
            nodes[i]["module"] = 20.1
        elif (nodes[i]["connection"] in type_21) :
            nodes[i]["module"] = 21.1
        elif (nodes[i]["connection"] in type_40) :
            nodes[i]["module"] = 40.1
        elif (nodes[i]["connection"] in type_45) :
            nodes[i]["module"] = 45.1

In [86]:
# # if there is a second floor sum add_dict's values with nodes' module types 
# # example: (node 0 has a type 20 module) and (and there will be a module on it) then (node 0's type = 20 + 3) 
# # if there is no second floor don't change anything
for k,i in nodes.items():
    if nodes[k]["module"]!=None:
        if (nodes[k]["floor"]==1):
            module=nodes[k]["module"]
            module=module+0.1
            nodes[k]["module"]=module

In [87]:
"""Exporting final nodes attributes and edge_dict used for orientation"""

file_nodes = "211024_nodes_final.txt"
file_edges = "211024_edge_dict.txt"

with open(file_nodes,"w") as nodes_outfile:
    try:
        json.dump(nodes, nodes_outfile)
        print(file_nodes + " has been updated successfully")
    except:
        print("Problem with updating file: ", file_nodes)

with open(file_edges,"w") as edges_outfile:
    try:
        json.dump(edge_dict, edges_outfile)
        print(file_edges + " has been updated successfully")
    except:
        print("Problem with updating file: ", file_edges)

211024_nodes_final.txt has been updated successfully
211024_edge_dict.txt has been updated successfully
